In [1]:
%connect_info

{
  "shell_port": 56993,
  "iopub_port": 56994,
  "stdin_port": 56995,
  "control_port": 56997,
  "hb_port": 56996,
  "ip": "127.0.0.1",
  "key": "6a0e1ba8-a7363069fe513001c4973398",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-1fdc1772-ae0c-4241-9936-503fd8427c99.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
%matplotlib inline

## Suggested models


`bert-base-multilingual-uncased`: (Original, not recommended) 12-layer, 768-hidden, 12-heads, 110M parameters. Trained on lower-cased text in the top 102 languages with the largest Wikipedia

`bert-base-multilingual-cased`: (New, recommended) 12-layer, 768-hidden, 12-heads, 110M parameters. Trained on cased text in the top 104 languages with the largest Wikipedias

`xlm-mlm-100-1280`: 16-layer, 1280-hidden, 16-heads XLM model trained with MLM (Masked Language Modeling) on 100 languages.


`distilbert-base-multilingual-cased`: 6-layer, 768-hidden, 12-heads, 134M parameters The multilingual DistilBERT model distilled from the Multilingual BERT model bert-base-multilingual-cased checkpoint.

In [3]:
import pickle
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from preprocess import load_dataset, Features, load_word_dataset

SEED = 42

In [4]:
train_files = ['data/GA/train.cupt']
dev_files = ['data/GA/train.cupt']

In [5]:
train_sentences, train_labels = load_word_dataset(train_files)
dev_sentences, dev_labels = load_word_dataset(dev_files)

In [6]:
len(train_sentences)

257

In [23]:
model_type = 'bert-base-multilingual-cased'

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_type)
model = TFAutoModel.from_pretrained(model_type)

In [22]:
model.summary()

Model: "tf_distil_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  134734080 
Total params: 134,734,080
Trainable params: 134,734,080
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
Total params: 177,853,440
Trainable params: 177,853,440
Non-trainable params: 0
_________________________________________________________________


In [44]:
input_ids = tokenizer.encode(
    train_sentences[0],
    add_special_tokens=True,
    return_tensors='tf')

In [45]:
input_ids.shape

TensorShape([1, 37])

In [46]:
output = model(input_ids)
output[0][0].numpy()[0,:].reshape(1, -1).shape

(1, 768)

In [49]:
output[0][0].numpy()[0, :].shape

(768,)

In [34]:
output[0].shape

TensorShape([37, 768])

In [17]:
# tf.keras.layers.GlobalAveragePooling1D()(output[0]).numpy().shape
output[0][1]

IndexError: tuple index out of range

In [12]:
model_type = 'bert-base-multilingual-cased'

In [13]:
code = 'DE'
with open('{}.{}.embdata.pkl'.format(code, model_type), 'rb') as f:
    de = pickle.load(f)

In [14]:
code = 'GA'
with open('{}.{}.embdata.pkl'.format(code, model_type), 'rb') as f:
    ga = pickle.load(f)

In [15]:
code = 'HI'
with open('{}.{}.embdata.pkl'.format(code, model_type), 'rb') as f:
    hi = pickle.load(f)

In [16]:
code = 'PT'
with open('{}.{}.embdata.pkl'.format(code, model_type), 'rb') as f:
    pt = pickle.load(f)

In [17]:
code = 'ZH'
with open('{}.{}.embdata.pkl'.format(code, model_type), 'rb') as f:
    zh = pickle.load(f)

In [18]:
data = {
    'DE': de,
    'GA': ga,
    'HI': hi,
    'PT': pt,
    'ZH': zh,    
}

In [19]:
with open('cls.{}.embdata.pkl'.format(model_type), 'wb') as f:
    pickle.dump(data, f)

In [17]:
import sys

In [18]:
sys.getsizeof(data)

248

In [21]:
(pt['x_train'].nbytes / 1024) / 1024

70.0341796875

In [16]:
len(de)

4

In [17]:
de.keys()

dict_keys(['x_train', 'y_train', 'x_dev', 'y_dev'])

In [22]:
de['x_train'].reshape(6568, -1).shape

ValueError: cannot reshape array of size 202662 into shape (6568,newaxis)